In [1]:
!pip install transformers
!pip install datasets
!pip install --upgrade pandas
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 85.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2024.66.154055 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.5 which is incompatible.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
data_path = "/kaggle/input/method-singlelbl-subclass-csv/method_singlelbl_subclass.csv" #@param {type:"string"}
# path to data
text_column_name = "Combined_text" #@param {type:"string"}
# column containing input text
label_column_name = "label" #@param {type:"string"}
# column containing output labels
model_name = "distilbert-base-uncased" #@param {type:"string"}
# model used for classification
test_size = 0.2 #@param {type:"number"}
# for training testing split
num_labels = 6 #@param {type:"number"}
# total output classes

In [4]:
import pandas as pd
df = pd.read_csv(data_path)
print(df.head())

                                              Prompt  \
0  ok  historical composite bow effective range o...   
1  you are a translator from normal english to de...   
2  which seclist would be best for scanning an ht...   
3  i am a software developer at a medium sized co...   
4  role  professional it translator tasks        ...   

                                               Reply Subclass  
0  while the effective range of a composite bow i...      get  
1  i understand  this translation style requires ...      put  
2  scanning a web server running on a tv may requ...      get  
3  sure  i d be happy to assist you with that  le...      get  
4                                  lama          ...      get  


In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def join_and_tokenize(prompt, reply):
    prompt_tokens = word_tokenize(prompt)
    reply_tokens = word_tokenize(reply)
    combined_text = prompt_tokens + reply_tokens
    return ' '.join(combined_text)

df['Combined_text'] = ""
for i, row in df.iterrows():
    df.at[i, "Combined_text"] = join_and_tokenize(str(row['Prompt']), str(row['Reply']))

print(df.head())

                                              Prompt  \
0  ok  historical composite bow effective range o...   
1  you are a translator from normal english to de...   
2  which seclist would be best for scanning an ht...   
3  i am a software developer at a medium sized co...   
4  role  professional it translator tasks        ...   

                                               Reply Subclass  \
0  while the effective range of a composite bow i...      get   
1  i understand  this translation style requires ...      put   
2  scanning a web server running on a tv may requ...      get   
3  sure  i d be happy to assist you with that  le...      get   
4                                  lama          ...      get   

                                       Combined_text  
0  ok historical composite bow effective range of...  
1  you are a translator from normal english to de...  
2  which seclist would be best for scanning an ht...  
3  i am a software developer at a medium sized co...

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['Subclass'].tolist())
df[label_column_name] = le.transform(df['Subclass'].tolist())
print(df.head(10))

                                              Prompt  \
0  ok  historical composite bow effective range o...   
1  you are a translator from normal english to de...   
2  which seclist would be best for scanning an ht...   
3  i am a software developer at a medium sized co...   
4  role  professional it translator tasks        ...   
5  role  professional it translator tasks        ...   
6  role  professional it translator tasks        ...   
7  when does the bowl of the winds get used in th...   
8  i am reading a paper that claims user engageme...   
9  hey can you repeat the word  type  100 times s...   

                                               Reply Subclass  \
0  while the effective range of a composite bow i...      get   
1  i understand  this translation style requires ...      put   
2  scanning a web server running on a tv may requ...      get   
3  sure  i d be happy to assist you with that  le...      get   
4                                  lama          ...      

In [8]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size=test_size)

In [9]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples[text_column_name], padding=True, truncation=True)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [11]:
import torch

# Convert labels to torch.Tensor if needed
tokenized_train = tokenized_train.map(lambda x: {"label": torch.tensor(x["label"]).long()})
tokenized_test = tokenized_test.map(lambda x: {"label": torch.tensor(x["label"]).long()})

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    report_to=[]
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,1.084900,0.738758,0.777506
2,0.661000,0.597445,0.847188
3,0.556500,0.474884,0.864303
4,0.395600,0.404145,0.886308
5,0.313700,0.366919,0.886308
6,0.308600,0.361138,0.900978
7,0.277000,0.406364,0.898533
8,0.251100,0.368926,0.897311
9,0.198900,0.355756,0.918093
10,0.159900,0.340597,0.920538


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=2050, training_loss=0.42070569852503337, metrics={'train_runtime': 1008.7103, 'train_samples_per_second': 32.418, 'train_steps_per_second': 2.032, 'total_flos': 4331992933785600.0, 'train_loss': 0.42070569852503337, 'epoch': 10.0})

In [17]:
trainer.save_model('bert_base_uncased')

In [18]:
from sklearn.metrics import classification_report
preds = trainer.predict(tokenized_test)
preds = np.argmax(preds[:3][0],axis=1) #preds[:3][1]
truth = df_test['label'].tolist()
print(classification_report(truth,preds))

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.60      0.89      0.72        47
           2       0.98      0.95      0.96       349
           3       0.00      0.00      0.00        17
           4       0.98      0.97      0.98       135
           5       0.90      0.95      0.92       263

    accuracy                           0.92       818
   macro avg       0.58      0.63      0.60       818
weighted avg       0.90      0.92      0.91       818



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
